In [1]:
import pandas as pd

# Reading in csv as panda df
df = pd.read_csv("0425_to_0703_ScientistRebellionCombinedTweets.csv")

# first rows
print(df.head())

#double check data type
print(type(df))

%whos

   Unnamed: 0         hashed_userid  masked_username location  following   
0           0  53819044136084617463         ****love      NaN          6  \
1           1  63365252551191839332         ****1721      NaN         83   
2           2  36591457752220856348        *****hayk   Earth         288   
3           3  36387613119620957083    *******nMedia      NaN       3642   
4           4  33033899708742110768  ********5338849      NaN        107   

   followers  totaltweets usercreateddt              tweetid   
0          1          180    2022-04-15  1518491125119877120  \
1         68          713    2021-06-23  1518491215641071619   
2      12276        43011    2009-04-10  1518491236147019776   
3       2640        15073    2009-02-23  1518491246137856001   
4         27          105    2022-04-12  1518491248214040576   

               tweetcreatedts  ...   
0  2022-04-25 07:24:31.000000  ...  \
1  2022-04-25 07:24:52.000000  ...   
2  2022-04-25 07:24:57.000000  ...   
3  202

In [2]:
pd.set_option('display.max_colwidth', 400)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print(list(df.columns))
print(df['is_retweet'].head(10))
print(df['language'].head(100))


['Unnamed: 0', 'hashed_userid', 'masked_username', 'location', 'following', 'followers', 'totaltweets', 'usercreateddt', 'tweetid', 'tweetcreatedts', 'retweetcount', 'text', 'hashtags', 'language', 'favorite_count', 'is_retweet', 'original_tweet_id', 'in_reply_to_status_id', 'is_quote_status', 'quoted_status_id', 'extractedts']
0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: is_retweet, dtype: bool
0      th
1     und
2      en
3      en
4      en
5      en
6      en
7      en
8      en
9      en
10     en
11     en
12     en
13     en
14     en
15     en
16     en
17    und
18     en
19     en
20     en
21     en
22     en
23    und
24     en
25     en
26     en
27     en
28    und
29     en
30     en
31    und
32     en
33     ja
34     en
35    und
36     en
37     en
38     en
39     en
40     en
41     en
42     en
43     en
44     en
45    und
46     en
47     en
48     en
49     en
50     en
51     en
52    und


In [3]:
# loading spacy requirements
import spacy
from spacy.lang.en import English
from spacy import displacy

# ensuring download of spacy's 'small' core english language model
!python -m spacy download en_core_web_sm

nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
# print the variable names for df
print(list(df.columns))

['Unnamed: 0', 'hashed_userid', 'masked_username', 'location', 'following', 'followers', 'totaltweets', 'usercreateddt', 'tweetid', 'tweetcreatedts', 'retweetcount', 'text', 'hashtags', 'language', 'favorite_count', 'is_retweet', 'original_tweet_id', 'in_reply_to_status_id', 'is_quote_status', 'quoted_status_id', 'extractedts']


In [5]:


# filtering for only english tweets and is_retweet == False
en_df = df[(df["language"] == "en") & (df["is_retweet"] == False)]






In [6]:
print(len(df[df["language"] == "en"])/len(df)*100)
print(len(en_df)/len(df[df["language"] == "en"])*100)
print(len(en_df)/len(df)*100)

# 78% of tweets are english
# 41% of english tweets are originals
# 31% of all tweets are english originals

77.98364885444965
41.03215490035814
31.99837159490915


In [7]:
## export sample of tweets to manually look over to identify hashtags
sample_tweets = en_df.sample(10000)
sample_tweets.to_csv("sample_tweets.csv", index=False)



In [8]:
# 
# heuristic_keywords = ['rebel', 'scientistrebel', 'extinct', 'scientist', 'crisis', 'climate', 'disobedi', 'protest', 'warming', 'extinct']

# en_df.loc[:, 'hashtag_any'] = en_df['hashtags'].apply(lambda x: any(keyword.lower() in str(x).lower() for keyword in heuristic_keywords))

# for keyword in heuristic_keywords:
#     en_df.loc[:, f'hashtag_{keyword}'] = en_df['hashtags'].apply(lambda x: keyword.lower() in str(x).lower())


In [9]:
from collections import Counter
import ast
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import spacy
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL
from spacy.util import minibatch, compounding

# Co-hashtags function
def find_co_hashtags(en_df, target_hashtag):
    co_hashtags = Counter()
    for index, row in en_df.iterrows():
        hashtags_list = eval(row['hashtags'])
        hashtag_texts = [hashtag['text'].lower() for hashtag in hashtags_list]
        if any(target_hashtag in ht for ht in hashtag_texts):
            for hashtag in hashtags_list:
                hashtag_text = hashtag['text'].lower()
                if target_hashtag not in hashtag_text:
                    co_hashtags[hashtag_text] += 1
    co_hashtags_df = pd.DataFrame(co_hashtags.most_common(), columns=['co_hashtag', 'count'])
    co_hashtags_df.to_csv(f"{target_hashtag}_cohashtags.csv", index=False)
    return co_hashtags_df

rebel_co_hashtags = find_co_hashtags(en_df, 'rebel')
extinct_co_hashtags = find_co_hashtags(en_df, 'extinct')

# Create weights dictionary
def create_weights_dict(df):
    total_count = df['count'].sum()
    weights = {}
    for index, row in df.iterrows():
        weights[row['co_hashtag']] = row['count'] / total_count
    return weights

extinct_weights = create_weights_dict(extinct_co_hashtags)
rebel_weights = create_weights_dict(rebel_co_hashtags)

# Calculate confidence score
def confidence_score(hashtags):
    base_score = 0
    certain_hashtags = ['scientistrebel', 'extinctionrebel']

    for hashtag_dict in hashtags:
        tag = hashtag_dict['text'].lower()
        if tag in certain_hashtags:
            return 1.0
        base_score += extinct_weights.get(tag, 0) + rebel_weights.get(tag, 0)

    transformed_score = np.log(1 + base_score * 10)
    max_score = np.log(1 + 10)
    normalized_score = transformed_score / max_score

    return min(normalized_score, 1.0)

en_df.loc[:, 'hashtags_list'] = en_df.loc[:,'hashtags'].apply(ast.literal_eval)
en_df.loc[:, 'confidence_score'] = en_df.loc[:,'hashtags_list'].apply(confidence_score)


/var/folders/vw/bpzpvrh5775b97vf11jmnf840000gn/T/ipykernel_1226/1130753148.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_df.loc[:, 'hashtags_list'] = en_df['hashtags'].apply(ast.literal_eval)
/var/folders/vw/bpzpvrh5775b97vf11jmnf840000gn/T/ipykernel_1226/1130753148.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_df.loc[:, 'confidence_score'] = en_df['hashtags_list'].apply(confidence_score)


In [11]:
# Train model to categorize text as rebel or not using the confidence score 

import spacy
from spacy.tokens import DocBin

def create_spacy_dataset(df, output_path):
    nlp = spacy.blank("en")
    doc_bin = DocBin()

    for _, row in df.iterrows():
        text = row["text"]
        score = row["confidence_score"]
        label = "HIT" if score >= 0.7 else "NOT_HIT"
        categories = {"HIT": score >= 0.7, "NOT_HIT": score < 0.7}

        doc = nlp.make_doc(text)
        doc.cats = categories
        doc_bin.add(doc)

    doc_bin.to_disk(output_path)

# Split your data into training and validation sets
train_df, dev_df = train_test_split(en_df, test_size=0.2, random_state=42)

# Create the .spacy files
create_spacy_dataset(train_df, "./train.spacy")
create_spacy_dataset(dev_df, "./dev.spacy")
